# THIS NOTEBOOK IS A WORK IN PROGRESS

# Notebook to add to Airtable from master CSV

## Proposed pipeline
Manual
1- Export directory listing Google Sheet into csv (need to download and re-upload it)

## Library imports

In [ ]:
!pip install airtable-python-wrapper
!pip install --upgrade gspread

  Found existing installation: gspread 3.0.1
    Uninstalling gspread-3.0.1:
      Successfully uninstalled gspread-3.0.1


In [ ]:
import os
import shutil
import glob
import csv
from datetime import datetime

from google.colab import drive
import pandas as pd

from airtable import Airtable

## Constants
The API keys should be hidden

In [ ]:
# Airtable API Key
API_KEY = 
# Airtable Base Key
BASE_KEY = 
# ProjectID to Airtable table mapping
TABLE_MAPPING = {
    "stress":"test-table",
    "bluestates":"bluestates",
    "redstates":"redstates",
    "bluecities":"bluecities",
    "swingstates":"swingstates"
    } 

# Folder path in GDrive to CSV
FULL_PATH_CSV = "/content/drive/My Drive/tech-volunteering/"

# Folder path in GDrive to VIDEO
FULL_PATH_CSV = "/content/drive/My Drive/tech-volunteering/"

# Master CSV file 
NAME_MASTER_CSV = "master.csv"

# File with directory listing and full_path
NAME_DIR_LISTING_CSV = "test-file-listing.csv"

# Columns to extract from directory listing
DIR_LISTING_COLUMNS = ["URL","Date"]

# Columns of interest to put in Airtable
AIRTABLE_COLUMNS = ["hash_id", "contributor_email", "contributor_name","time_uploaded",
           "shot_title", "caption", "media_type", "latitude", "longitude", 
           "video_duration_seconds", "video_duration_minutes", "tags", "url_drive", "time_uploaded_drive"]

# List/tuple of possible file format for the videos
LIST_VIDEO_FORMATS = (".mov", ".mp4")

## Google Collab mounting
Need to accept token

In [ ]:
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


## Functions

In [ ]:
def convert_to_ISO8601(raw_date):
  try:
    formatted_date = datetime.strptime(raw_date, "%Y-%m-%d %H:%M:%S").isoformat()
  except:
    formatted_date = datetime.strptime(raw_date, "%m/%d/%Y %H:%M:%S").isoformat()

  return formatted_date

def processed_csv_airtable(df_raw,airtable_instance):

  # Record hashid that is not processed
  error_hashid = []

  # Iterate through each row
  for i, row in df_raw.iterrows():
    hashid = row["hash_id"]
    try: 
      # Transform into ISO 8601 formatted date
      row["time_uploaded"] = convert_to_ISO8601(row["time_uploaded"])

      # Split video duration into int minutes and int seconds
      row["video_duration"] = row["video_duration"].split(":")
      row["video_duration_minutes"] = int(row["video_duration"][0]) * 60 + int(row["video_duration"][1])
      row["video_duration_seconds"] = int(row["video_duration"][2])

      # Convert latitude to float
      row["latitude"] = float(row["latitude"])

      # Convert longtitude to float
      row["longitude"] = float(row["longitude"])

      # Assuming the tags are always separated by newlines
      row["tags"] = row["tags"].replace("\n", " ") # Ensures that multiple occurences of \n are replaced by whitespaces
      row["tags"] = row["tags"].split() # Split into seperate tags

      # URL in Drive
      row["url_drive"] = row["URL"]

      # Date uploaded
      row["time_uploaded_drive"] = convert_to_ISO8601(row["Date"])

      # Obtain subset to insert into Airtable
      subset= row[AIRTABLE_COLUMNS ]

      dict_subset = subset.to_dict()

      airtable_instance.insert(dict_subset, typecast=True)

    except Exception as e:
          print("\n")
          print(str(e))
          print("Unable to process: " + hashid)
          error_hashid.append((hashid, str(e)))
          continue
  return error_hashid

## CSV processing


In [ ]:
# From which row to start adding from master csv
starting_row = 0


In [ ]:
# From which row to start adding from master csv
starting_row = 0

# Read CSV of directory listing
df_dir_list = pd.read_csv(FULL_PATH_CSV + NAME_DIR_LISTING_CSV)

# Only subset the video files 
df_dir_list_subset = df_dir_list[df_dir_list["Name"].str.endswith(LIST_VIDEO_FORMATS)]

# Add column for hashIDs
df_dir_list_subset["hash_id"] =  df_dir_list_subset['Name'].apply(lambda filename: filename[:-4].split("_")[-1])

# Make it ready to merge with master csv via hash_id
df_dir_list_subset_for_merge = df_dir_list_subset[DIR_LISTING_COLUMNS + ["hash_id"]]

# Read master csv
df_master_csv = pd.read_csv(FULL_PATH_CSV+NAME_MASTER_CSV,skiprows=starting_row )

# Merge
df_merged = df_master_csv.merge(df_dir_list_subset_for_merge, how="left", on="hash_id")

# Unique project id's
list_unique_project_id = list(df_merged["project_id"].unique())

# For each project id
for project_id in list_unique_project_id:
  df_merged_subset = df_merged[df_merged["project_id"] == project_id]
  table_name = TABLE_MAPPING[project_id]
  # Authentificate in Airtable
  try:
    airtable_instance_project = Airtable(BASE_KEY, table_name, api_key=API_KEY)
    print("\n Airtable authenticated \n")
  except:
      print("Problem with Airtable authentication")

  print("Adding to table " + table_name + "\n")
  # Add to Airtable
  list_errors = processed_csv_airtable(df_merged_subset,airtable_instance_project)

  # What to do with list of errors?
  


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  



 Airtable authenticated 

Adding to table test-table



# Update Airtable and Directory

## Step 1

Authentication

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
import os
import shutil
import glob
import csv
from datetime import datetime

from google.colab import drive
import pandas as pd

from airtable import Airtable

drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


## Step 2

Setup

In [ ]:
# AIRTABLE SETUP

# Airtable API Key
API_KEY = 
# Airtable Base Key
BASE_KEY = 
# ProjectID to Airtable table mapping
TABLE_MAPPING = {
    "stress":"test-table",
    "bluestates":"bluestates",
    "redstates":"redstates",
    "bluecities":"bluecities",
    "swingstates":"swingstates"
    } 

# PATHS

# Folder path in GDrive to videos
FULL_PATH_VIDEO = "/content/drive/My Drive/Protect the Results Story Team/10 PTR DISTRIBUTION/Footage Library/"

# Folder path in GDrive to CSV
FULL_PATH_CSV = "/content/drive/My Drive/Protect the Results Story Team/10 PTR DISTRIBUTION/Footage Library/METADATA/"

# FILES

# Master CSV file 
NAME_MASTER_CSV = "master_csv_test.csv"

# File with directory listing and full_path
NAME_DIR_LISTING_CSV = "video_directory.csv"

# OTHER CONSTANTS

# Columns to extract from directory listing
DIR_LISTING_COLUMNS = ["URL","Date"]

# Columns of interest to put in Airtable
AIRTABLE_COLUMNS = ["hash_id", "contributor_email", "contributor_name","time_uploaded",
           "shot_title", "caption", "media_type", "latitude", "longitude", 
           "video_duration_seconds", "video_duration_minutes", "tags", "url_drive", "time_uploaded_drive"]

# List/tuple of possible file format for the videos
LIST_VIDEO_FORMATS = (".mov", ".mp4")

# FUNCTIONS

def convert_to_ISO8601(raw_date):
  try:
    formatted_date = datetime.strptime(raw_date, "%Y-%m-%d %H:%M:%S").isoformat()
  except:
    formatted_date = datetime.strptime(raw_date, "%m/%d/%Y %H:%M:%S").isoformat()

  return formatted_date

def processed_csv_airtable(df_raw,airtable_instance):

  # Record hashid that is not processed
  error_hashid = []

  # Iterate through each row
  for i, row in df_raw.iterrows():
    hashid = row["hash_id"]
    try: 
      # Transform into ISO 8601 formatted date
      row["time_uploaded"] = convert_to_ISO8601(row["time_uploaded"])

      # Split video duration into int minutes and int seconds
      row["video_duration"] = row["video_duration"].split(":")
      row["video_duration_minutes"] = int(row["video_duration"][0]) * 60 + int(row["video_duration"][1])
      row["video_duration_seconds"] = int(row["video_duration"][2])

      # Convert latitude to float
      row["latitude"] = float(row["latitude"])

      # Convert longtitude to float
      row["longitude"] = float(row["longitude"])

      # Assuming the tags are always separated by newlines
      row["tags"] = row["tags"].replace("\n", " ") # Ensures that multiple occurences of \n are replaced by whitespaces
      row["tags"] = row["tags"].split() # Split into seperate tags

      # URL in Drive
      row["url_drive"] = row["URL"]

      # Date uploaded
      row["time_uploaded_drive"] = convert_to_ISO8601(row["Date"])

      # Obtain subset to insert into Airtable
      subset= row[AIRTABLE_COLUMNS ]

      dict_subset = subset.to_dict()

      airtable_instance.insert(dict_subset, typecast=True)

    except Exception as e:
          print("\n")
          print(str(e))
          print("Unable to process: " + hashid)
          error_hashid.append((hashid, str(e)))
          continue
  return error_hashid

In [ ]:
# read in the existing google sheet directory as a pandas dataframe
directory = gc.open("project_directory").sheet1

rows = directory.get_all_values()

import pandas as pd
df_dir_list = pd.DataFrame.from_records(rows)

# need to replace column names with first row
new_header = df_dir_list.iloc[0] 
df_dir_list = df_dir_list[1:] 
df_dir_list.columns = new_header 

In [ ]:
df_dir_list

,Full Path,Name,Date,URL,Last Updated,Description,Size
1,Footage Library/selects,selects,11/4/2020 12:21:48,https://drive.google.com/drive/folders/1w62_GR...,11/4/2020 12:21:42,,0
2,Footage Library/METADATA,METADATA,11/4/2020 12:21:46,https://drive.google.com/drive/folders/1gXaNnx...,11/4/2020 12:21:38,,0
3,Footage Library/METADATA/processed_logs,processed_logs,11/4/2020 12:22:53,https://drive.google.com/drive/folders/1xAkEl9...,11/4/2020 12:22:46,,0
4,Footage Library/METADATA/processed_csv,processed_csv,11/4/2020 12:22:50,https://drive.google.com/drive/folders/1806TYd...,11/4/2020 12:22:42,,0
5,Footage Library/METADATA/incoming_csv,incoming_csv,11/4/2020 12:22:45,https://drive.google.com/drive/folders/1S-ogqd...,11/4/2020 12:22:38,,0
...,...,...,...,...,...,...,...
415,Footage Library/swingstates/trim/swingstates_C...,swingstates_Cindy.burstein@gmail.com_201104_NO...,11/4/2020 13:21:51,https://drive.google.com/file/d/1ZPtaVPgJDBueh...,11/4/2020 21:09:36,,14618645
416,Footage Library/swingstates/trim/swingstates_H...,swingstates_Hillary_201104_mqZqGAQWXfm5.mov,11/4/2020 13:01:44,https://drive.google.com/file/d/1O0YWqiensqxO9...,11/4/2020 20:55:12,,64602037
417,Footage Library/swingstates/trim/swingstates_C...,swingstates_Cindy.burstein@gmail.com_201104_lR...,11/4/2020 13:02:25,https://drive.google.com/file/d/18PnCasDrS7QX9...,11/4/2020 13:28:53,,16298093
418,Footage Library/swingstates/sensitive,sensitive,11/3/2020 10:27:40,https://drive.google.com/drive/folders/1nS9b1A...,11/3/2020 9:39:56,,0


## Step 3

Execution

In [ ]:
# From which row to start adding from master csv
starting_row = 0

# Only subset the video files 
df_dir_list_subset = df_dir_list[df_dir_list["Name"].str.endswith(LIST_VIDEO_FORMATS)]

# Export directory of videos
df_dir_list_subset.to_csv(FULL_PATH_VIDEO + NAME_DIR_LISTING_CSV)

# Add column for hashIDs
df_dir_list_subset["hash_id"] =  df_dir_list_subset['Name'].apply(lambda filename: filename[:-4].split("_")[-1])

# Make it ready to merge with master csv via hash_id
df_dir_list_subset_for_merge = df_dir_list_subset[DIR_LISTING_COLUMNS + ["hash_id"]]

# Read master csv
df_master_csv = pd.read_csv(FULL_PATH_CSV+NAME_MASTER_CSV,skiprows=starting_row )

# Merge
df_merged = df_master_csv.merge(df_dir_list_subset_for_merge, how="left", on="hash_id")

# Unique project id's
list_unique_project_id = list(df_merged["project_id"].unique())

# For each project id
for project_id in list_unique_project_id:
  df_merged_subset = df_merged[df_merged["project_id"] == project_id]
  table_name = TABLE_MAPPING[project_id]
  # Authentificate in Airtable
  try:
    airtable_instance_project = Airtable(BASE_KEY, table_name, api_key=API_KEY)
    print("\n Airtable authenticated \n")
  except:
      print("Problem with Airtable authentication")

  print("Adding to table " + table_name + "\n")
  # Add to Airtable
  list_errors = processed_csv_airtable(df_merged_subset,airtable_instance_project)

  # What to do with list of errors?

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()



 Airtable authenticated 

Adding to table bluestates



strptime() argument 1 must be str, not float
Unable to process: LZ9ZWOwP0t4N


strptime() argument 1 must be str, not float
Unable to process: BrBr97xq3U38


strptime() argument 1 must be str, not float
Unable to process: OgYg6XGr4Fvo


strptime() argument 1 must be str, not float
Unable to process: XQDQkx5nghqL

 Airtable authenticated 

Adding to table swingstates



('404 Client Error: Not Found for url: https://api.airtable.com/v0/appNVrKGAEP2sHXUw/swingstates', "404 Client Error: Not Found for url: https://api.airtable.com/v0/appNVrKGAEP2sHXUw/swingstates [Error: {'type': 'TABLE_NOT_FOUND', 'message': 'Could not find table swingstates in application appNVrKGAEP2sHXUw'}]")
Unable to process: NOkOXl06WF9M


('404 Client Error: Not Found for url: https://api.airtable.com/v0/appNVrKGAEP2sHXUw/swingstates', "404 Client Error: Not Found for url: https://api.airtable.com/v0/appNVrKGAEP2sHXUw/swingstates [Error: {'type': 'TABLE_NOT_